In [7]:
'''Part of code contributed by Chris'''
import numpy as np
import pandas as pd
import sklearn

fullData = pd.read_csv('/Users/haowei/Documents/gal/Day37_case_study2/case_study2/Churn/churn_train.csv')
fullData.head()


fullData['last_trip_date'] = pd.to_datetime(fullData['last_trip_date'])
condition = fullData['last_trip_date'] < '2014-06-01'
fullData['churn'] = 1
fullData.ix[~condition, 'churn'] = 0
fullData.head(50)

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,churn
0,6.94,5.0,5.0,1.00,Astapor,2014-05-03,Android,2014-01-12,0.0,0,False,100.0,1
1,8.06,5.0,5.0,1.00,Astapor,2014-01-26,Android,2014-01-25,0.0,2,True,0.0,1
2,21.50,4.0,NaN,1.00,Winterfell,2014-05-21,iPhone,2014-01-02,0.0,1,True,100.0,1
3,9.46,5.0,NaN,2.75,Winterfell,2014-01-10,Android,2014-01-09,100.0,1,False,100.0,1
4,13.77,5.0,NaN,1.00,Winterfell,2014-05-13,iPhone,2014-01-31,0.0,0,False,100.0,1
5,14.51,5.0,NaN,1.00,Astapor,2014-04-22,iPhone,2014-01-29,0.0,0,True,100.0,1
6,4.87,4.7,4.0,1.00,Astapor,2014-04-21,Android,2014-01-18,0.0,0,False,66.7,1
7,1.78,4.0,5.0,1.00,Winterfell,2014-01-12,iPhone,2014-01-11,0.0,3,False,0.0,1
8,6.87,5.0,4.7,1.00,Astapor,2014-01-30,Android,2014-01-01,0.0,3,False,66.7,1
9,2.41,5.0,3.0,1.00,Winterfell,2014-01-29,iPhone,2014-01-28,0.0,1,False,100.0,1


In [14]:
def crossVal(x_train, y_train, model, fold =5 ):
    #x_train, y_train should be numpy arrary
    # modle is a test modle, e.g., #sklearn.linear_model.LinearRegression()
    kf = sklearn.cross_validation.KFold(len(x_train), fold) 
    mses = []
    for train_ind, test_ind in kf:
        train_x = x_train[train_ind]
        train_y = y_train[train_ind]
        test_x = x_train[test_ind] # for the current q, we only use x_train for KFold
        test_y = y_train[test_ind]
        
        md = model.fit(train_x, train_y)#sklearn.linear_model.LinearRegression().fit(train_x, train_y)
        y_hat = md.predict(test_x)
        mse = sklearn.metrics.mean_squared_error(test_y, y_hat)
        mses.append(mse)
    return np.mean(mses)

fullData['city'].unique()
df_city = pd.get_dummies(fullData['city'])
df_city.head()

,Astapor,King's Landing,Winterfell
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0
